# SPSS Modelerの回帰モデル評価のための精度分析ノード:回帰をPythonで書き換える。

SPSS Modeler ノードリファレンス 6-4b 精度分析:回帰 - Qiita 

https://qiita.com/kawada2017/items/bc543f0f1bb401854c0e

# データの読み込み

In [1]:
import pandas as pd
#!wget 'https://raw.githubusercontent.com/yoichiro0903n/blue/main/cars.csv'
df=pd.read_csv('cars.csv')
df

,燃費,エンジン,馬力,車体重量,加速性,年式,生産国,気筒数
0,19.0,70,97,2330,13.5,72.0,日本,3気筒
1,18.0,70,90,2124,13.5,73.0,日本,3気筒
2,21.5,80,110,2720,13.5,77.0,日本,3気筒
3,23.7,70,100,2420,12.5,80.0,日本,3気筒
4,20.3,131,103,2830,15.9,78.0,ヨーロッパ,5気筒
...,...,...,...,...,...,...,...,...
387,22.0,146,97,2815,14.5,77.0,日本,6気筒
388,32.7,168,132,2910,11.4,80.0,日本,6気筒
389,25.4,168,116,2900,12.6,81.0,日本,6気筒
390,24.2,146,120,2930,13.8,81.0,日本,6気筒


# 1sk．全データに基づく回帰モデルの精度分析　scikit-learn

In [2]:
# モデルの作成

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split


# 説明変数
X = df[['エンジン']]
# 目的変数
Y = df['馬力']


# 線形回帰のモデルを作成
lr = LinearRegression()
lr.fit(X, Y)


LinearRegression()

In [3]:
#回帰モデルの評価
def analysis_eval(X, Y, lr):
    Y_pred = lr.predict(X)  # 学習データに対して目的変数を予測
    df_residual = pd.DataFrame(data={'Y': Y, 'Y_pred': Y_pred})
    df_residual['残差'] = df_residual['Y']-df_residual['Y_pred']
    dic_analyis_eval = {}
    #最小誤差
    #print('最小誤差: ', df_residual['残差'].min())
    dic_analyis_eval['最小誤差'] = df_residual['残差'].min()

    #最大誤差を出力
    from sklearn.metrics import max_error
    #print('最大誤差: ', max_error(Y, Y_pred))
    #dic_analyis_eval['最大誤差'] = df_residual['残差'].max()
    dic_analyis_eval['最大誤差'] = max_error(Y, Y_pred)

    #平均誤差
    #print('平均誤差: ', df_residual['残差'].mean())
    dic_analyis_eval['平均誤差'] = df_residual['残差'].mean()

    #絶対平均誤差(MAE)
    from sklearn.metrics import mean_absolute_error
    #print('絶対平均誤差(MAE): ', mean_absolute_error(Y, Y_pred))
    dic_analyis_eval['絶対平均誤差(MAE)'] = mean_absolute_error(Y, Y_pred)

    #二乗平均平方根誤差 (RMSE)
    from sklearn.metrics import mean_squared_error
    import numpy as np
    #print('二乗平均平方根誤差 (RMSE): ', np.sqrt(mean_squared_error(Y, Y_pred)))
    dic_analyis_eval['二乗平均平方根誤差 (RMSE)'] = np.sqrt(
        mean_squared_error(Y, Y_pred))

    #線形相関
    #print('線形相関: ', df_residual[['Y','Y_pred']].corr().at['Y', 'Y_pred'])
    dic_analyis_eval['線形相関'] = df_residual[[
        'Y', 'Y_pred']].corr().at['Y', 'Y_pred']

    #頻度数
    #print('頻度数: ', len(df_residual))
    dic_analyis_eval['頻度数'] = len(df_residual)

    #平均絶対パーセント誤差（MAPE）
    from sklearn.metrics import mean_absolute_percentage_error
    #print('平均絶対パーセント誤差（MAPE）: ', np.sqrt(mean_absolute_percentage_error(Y, Y_pred)))
    dic_analyis_eval['平均絶対パーセント誤差（MAPE）'] = mean_absolute_percentage_error(Y, Y_pred)

    return dic_analyis_eval


In [4]:
analysis_eval(X,Y,lr)

{'最小誤差': -59.57619761558347,
 '最大誤差': 76.01978973370788,
 '平均誤差': 8.700523295021635e-15,
 '絶対平均誤差(MAE)': 12.932339595462782,
 '二乗平均平方根誤差 (RMSE)': 17.852878968271614,
 '線形相関': 0.8732150534421105,
 '頻度数': 392,
 '平均絶対パーセント誤差（MAPE）': 0.13858786059996991}

scikit-learn で回帰モデルの結果を評価する – Python でデータサイエンス https://pythondatascience.plavox.info/scikit-learn/%E5%9B%9E%E5%B8%B0%E3%83%A2%E3%83%87%E3%83%AB%E3%81%AE%E8%A9%95%E4%BE%A1

API Reference — scikit-learn 1.1.2 documentation https://scikit-learn.org/stable/modules/classes.html#regression-metrics

# 1st．全データに基づく回帰モデルの精度分析　statsmodels

In [5]:
Y_pred = lr.predict(X)
from statsmodels.tools.eval_measures import maxabs
print('最大誤差: ', maxabs(Y, Y_pred))
from statsmodels.tools.eval_measures import meanabs
print('絶対平均誤差(MAE): ', meanabs(Y, Y_pred)) 
from statsmodels.tools.eval_measures import rmse
print('二乗平均平方根誤差 (RMSE): ', rmse(Y, Y_pred))

最大誤差:  76.01978973370788
絶対平均誤差(MAE):  12.932339595462782
二乗平均平方根誤差 (RMSE):  17.852878968271614


# 2sk．学習データとテストデータに分けた回帰モデルの精度分析　sklearn

In [6]:
#学習とテストデータに分割
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.5, random_state=1234567
)

# 線形回帰のモデルを作成
lr2 = LinearRegression()
lr2.fit(X_train, Y_train)
# 各モデルの評価を算出
analysis_eva_train=analysis_eval(X_train,Y_train,lr2)
analysis_eva_test=analysis_eval(X_test,Y_test,lr2)
# 整形して表示
pd.options.display.float_format = '{:.3f}'.format
pd.DataFrame([analysis_eva_train,analysis_eva_test],index=['学習','テスト']).T

,学習,テスト
最小誤差,-42.994,-59.502
最大誤差,67.937,77.275
平均誤差,-0.000,-0.146
絶対平均誤差(MAE),13.072,12.930
二乗平均平方根誤差 (RMSE),17.290,18.468
線形相関,0.867,0.879
頻度数,196.000,196.000
平均絶対パーセント誤差（MAPE）,0.136,0.143


Tools — statsmodels https://www.statsmodels.org/dev/tools.html

In [7]:
import sys

print(sys.version)
print(pd.__version__)
import sklearn
print(sklearn.__version__)
import statsmodels
print(statsmodels.__version__)

3.8.10 (tags/v3.8.10:3d8993a, May  3 2021, 11:48:03) [MSC v.1928 64 bit (AMD64)]
1.4.1
1.1.1
0.13.2
